In [1]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import glob
import os
import datetime

In [2]:
def load_data():

    train = []

    for img in glob.glob("training/hi/*.jpg"):
        n = cv2.imread(img)
        np.squeeze(n)
        train.append([n,1])


    for img in glob.glob("training/background/*.jpg"):
        n = cv2.imread(img)
        np.squeeze(n)
        train.append([n,0])


    # test = []

    # for img in glob.glob("test/*.jpg"):
    #     n = cv2.imread(img)
    #     np.squeeze(n)
    #     test.append([n,0])
    train = np.array(train)

    np.random.shuffle(train)


    x_train = train[:5000,0]
    y_train = train[:5000,1]

    x_test = train[5000:,0]
    y_test = train[5000:,1]


    X = []
    XT = []
    #X.append(x_train)
    for i in range(len(x_train)):
      X.append(x_train[i])

    X = np.array(X)
    X  = X/255.0
    Y = np.array(y_train)

    # #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    for i in range(len(x_test)):
      XT.append(x_test[i])

    XT = np.array(XT)
    XT  = XT/255.0
    YT = np.array(y_test)
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

    print(X.shape)
    print(Y.shape)

    X = tf.convert_to_tensor(X,dtype=tf.float32)
    Y = tf.convert_to_tensor(Y,dtype=tf.float32)

    XT = tf.convert_to_tensor(XT,dtype=tf.float32)
    YT = tf.convert_to_tensor(YT,dtype=tf.float32)


    return X,Y,XT,YT


In [81]:
X,Y,XT,YT = load_data()


model = tf.keras.models.Sequential()
model.add(layers.Conv2D(128,3,input_shape=(64,64,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64,3,activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32,3,activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(16,3,activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(100,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.summary()

model.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])
import os
import datetime
logdir = os.path.join("logs", "batch_10_iteration_200")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

<ipython-input-2-f624c3db6840>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train = np.array(train)


(5000, 64, 64, 3)
(5000,)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
batch_normalization_48 (Batc (None, 31, 31, 128)       512       
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 29, 29, 64)        73792     
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_49 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_50 (Conv2D)         

In [10]:
print(tf.__version__)
print(tf.config.list_physical_devices())

2.5.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [82]:
model.fit(X,Y,epochs=200,validation_data=(XT,YT),batch_size=10,callbacks=[tensorboard_callback])
model.save('./MODEL/batch_10_iteration_200')

Epoch 1/200
500/500 [==============================] - 17s 19ms/step - loss: 0.4195 - accuracy: 0.8200 - val_loss: 0.6013 - val_accuracy: 0.7794
Epoch 2/200
500/500 [==============================] - 7s 14ms/step - loss: 0.3028 - accuracy: 0.8862 - val_loss: 0.1978 - val_accuracy: 0.9223
Epoch 3/200
500/500 [==============================] - 7s 15ms/step - loss: 0.2620 - accuracy: 0.9024 - val_loss: 0.1690 - val_accuracy: 0.9406
Epoch 4/200
500/500 [==============================] - 7s 13ms/step - loss: 0.2254 - accuracy: 0.9126 - val_loss: 0.1332 - val_accuracy: 0.9566
Epoch 5/200
500/500 [==============================] - 7s 14ms/step - loss: 0.2050 - accuracy: 0.9230 - val_loss: 0.1050 - val_accuracy: 0.9680
Epoch 6/200
500/500 [==============================] - 7s 15ms/step - loss: 0.1826 - accuracy: 0.9328 - val_loss: 0.1034 - val_accuracy: 0.9714
Epoch 7/200
500/500 [==============================] - 7s 14ms/step - loss: 0.1520 - accuracy: 0.9446 - val_loss: 0.1332 - val_accuracy

500/500 [==============================] - 7s 15ms/step - loss: 0.0242 - accuracy: 0.9916 - val_loss: 0.0279 - val_accuracy: 0.9943
Epoch 58/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0187 - accuracy: 0.9930 - val_loss: 0.0859 - val_accuracy: 0.9771
Epoch 59/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0210 - accuracy: 0.9936 - val_loss: 0.0295 - val_accuracy: 0.9874
Epoch 60/200
500/500 [==============================] - 7s 15ms/step - loss: 0.0170 - accuracy: 0.9936 - val_loss: 0.0278 - val_accuracy: 0.9886
Epoch 61/200
500/500 [==============================] - 7s 14ms/step - loss: 0.0171 - accuracy: 0.9944 - val_loss: 0.0453 - val_accuracy: 0.9909
Epoch 62/200
500/500 [==============================] - 6s 13ms/step - loss: 0.0174 - accuracy: 0.9944 - val_loss: 0.0634 - val_accuracy: 0.9817
Epoch 63/200
500/500 [==============================] - 7s 15ms/step - loss: 0.0125 - accuracy: 0.9960 - val_loss: 0.0273 - val_accuracy: 0.992

500/500 [==============================] - 7s 13ms/step - loss: 0.0142 - accuracy: 0.9960 - val_loss: 0.0166 - val_accuracy: 0.9943
Epoch 114/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0101 - accuracy: 0.9962 - val_loss: 0.0258 - val_accuracy: 0.9931
Epoch 115/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0100 - accuracy: 0.9960 - val_loss: 0.0198 - val_accuracy: 0.9920
Epoch 116/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0194 - accuracy: 0.9950 - val_loss: 0.0198 - val_accuracy: 0.9920
Epoch 117/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0083 - accuracy: 0.9962 - val_loss: 0.0631 - val_accuracy: 0.9874
Epoch 118/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0102 - accuracy: 0.9962 - val_loss: 0.0293 - val_accuracy: 0.9909
Epoch 119/200
500/500 [==============================] - 7s 14ms/step - loss: 0.0089 - accuracy: 0.9970 - val_loss: 0.0485 - val_accuracy:

500/500 [==============================] - 7s 14ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 0.0306 - val_accuracy: 0.9920
Epoch 170/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0124 - accuracy: 0.9970 - val_loss: 0.0338 - val_accuracy: 0.9909
Epoch 171/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0089 - accuracy: 0.9960 - val_loss: 0.0361 - val_accuracy: 0.9909
Epoch 172/200
500/500 [==============================] - 6s 13ms/step - loss: 0.0071 - accuracy: 0.9982 - val_loss: 0.0465 - val_accuracy: 0.9863
Epoch 173/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0066 - accuracy: 0.9978 - val_loss: 0.0138 - val_accuracy: 0.9920
Epoch 174/200
500/500 [==============================] - 6s 13ms/step - loss: 0.0101 - accuracy: 0.9962 - val_loss: 0.0291 - val_accuracy: 0.9897
Epoch 175/200
500/500 [==============================] - 7s 13ms/step - loss: 0.0151 - accuracy: 0.9956 - val_loss: 0.0251 - val_accuracy:

In [12]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [83]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 21020), started 5:56:38 ago. (Use '!kill 21020' to kill it.)

In [84]:
test = []

for img in glob.glob("test/*.jpg"):
        n = cv2.imread(img)
        np.squeeze(n)
        test.append(n)
        
test = np.array(test)
test = test/255.0

print(test.shape)

Model = tf.keras.models.load_model('MODEL/batch_10_iteration_200')
        
prediction  = Model.predict(test)
# print(prediction)

for i in range(len(prediction)):
    if(prediction[i]>0.5):
        prediction[i]=1
    else:
        prediction[i]=0
        
print(len(prediction))      



(98, 64, 64, 3)
98


In [85]:
import random
import json
# from utils.io import write_json

def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

def read_json(filename):
    with open(filename, 'r') as outfile:
        data =  json.load(outfile)
    return data

def generate_sample_file(filename):
    res = {}
    for i in range(1,99):
        test_set = str(i) + '.png'
        res[test_set] = int(prediction[i-1])
    write_json(filename, res)
    

generate_sample_file("result")    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
